In [1]:
import Classfier
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.metrics import f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from torchvision import models
from sklearn.manifold import TSNE
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import os
from PIL import Image
import csv
import time 
import random  
import numpy as np
import zipfile

In [2]:
os.makedirs("results", exist_ok=True)

In [3]:
Classfier.set_random_seed(42)

In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),  
    transforms.ToTensor(), 
])

train_dataset = Classfier.CustomDataset(root_dir='D:/jupyternotebook/X-1/CWT-IL-ACSAWGAN-GP/Comparison Models/DSEA-SMOTE/BR1_400/images', transform=transform)
test_dataset = Classfier.CustomDataset(root_dir='D:/jupyternotebook/X-1/CWT-IL-ACSAWGAN-GP/Data/CWRU/test_set', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [5]:
patience = 5
early_stopping = Classfier.EarlyStopping(patience=patience, min_loss=0.0001, max_accuracy=0.995, verbose=True)

In [7]:
class_names = [str(i) for i in range(10)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Classfier.VGG16().to(device)  
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001) # 假设你已经准备好了模型、训练数据加载器、损失函数和优化器


In [ ]:
train_losses, train_accuracies, train_times = Classfier.train(
    model, train_loader, criterion, optimizer, 
    num_epochs=200, early_stopping=early_stopping, 
    device='cuda', output_file='results/training_results.csv'
)

D:\Anaconda3\envs\dl\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/200], Loss: 1.8204, Accuracy: 27.02%, Time: 233.19s
Epoch [2/200], Loss: 0.3572, Accuracy: 88.21%, Time: 68.62s


In [ ]:
Classfier.plot_loss_acc_time_curve(train_losses, train_accuracies, train_times,  plot_dir='results')

In [ ]:
Classfier.test_model_with_metrics(model, test_loader, class_names,device)

In [ ]:
layer_name = model.classifier[-2]  
hook = layer_name.register_forward_hook(hook_fn)

class_names = [str(i) for i in range(10)]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features_tsne, labels = Classfier.get_vgg16_features_and_tsne(model, test_loader, device)
Classfier.plot_tsne(features_tsne, labels, class_names,save_path="results/tsne_visualization.png")

In [ ]:
folder_to_zip = 'results'  
zip_filename = 'results.zip'  
Classfier.zip_folder(folder_to_zip, zip_filename)